In [1]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib import RDF, RDFS, OWL, XSD
import sys
sys.path.append('../')

sys.path.append('../andre')
import andre.utils as utils
from andre.utils import schema as SCHEMA
import collections

In [2]:
publishers_btlf = Graph()
publishers_btlf.bind("schema","http://schema.org/", override=True, replace=True)
publishers_btlf.parse("../final_datasets/btlf_books/Graphes/grapheEditeurs_BTLF.ttl")
len(list(publishers_btlf))

1486

In [3]:
list(publishers_btlf)[1303]

(rdflib.term.URIRef('http://schema.org/publisher618'),
 rdflib.term.URIRef('http://schema.org/name'),
 rdflib.term.Literal('Editions Poisson rouge.Oi'))

In [4]:
publishers_bnf_constellations = Graph()
publishers_bnf_constellations.bind("schema","http://schema.org/", override=True, replace=True)
publishers_bnf_constellations.parse("../final_datasets/publishers.ttl")
len(list(publishers_bnf_constellations))

4690

In [5]:
publishers_bnf_constellations_dict = {}
doublons = []
for publisher in publishers_bnf_constellations.subjects(RDF.type, SCHEMA.Publisher):
    raw_name = str(publishers_bnf_constellations.value(publisher, SCHEMA.name))
    preprocessed_name = utils.preprocess_publisher_name(raw_name)

    
    if preprocessed_name in publishers_bnf_constellations_dict.keys():
        # print("doublons", name)
        doublons.append(preprocessed_name)
    else:
        publishers_bnf_constellations_dict[preprocessed_name] = utils.Publisher(source="bnf_constellations", 
                                                                        uri=publisher, 
                                                                        raw_name=raw_name, 
                                                                        preprocessed_name=preprocessed_name)

print(len(doublons))
len(publishers_bnf_constellations_dict)

58


2237

In [6]:
doublons

['memo',
 'ekare',
 'denoel',
 'lelievredemars',
 'palette',
 'disneyhyperionbooks',
 'petitsgenies',
 'delachauxetniestle',
 'lapoulequipond',
 'northsouthbooks',
 'motus',
 'quatrefleuves',
 'format',
 '123soleil',
 'ruedelechiquier',
 'eyrolles',
 'ceres',
 'larche',
 'pika',
 'etre',
 'grenouille',
 'duberyl',
 'ankama',
 'duchene',
 'daralsaqi',
 'flammarion',
 'xyz',
 'grund',
 'bamboo',
 'lansman',
 'daraladab',
 'edicef',
 'dupepin',
 'pemf',
 'zebulo',
 'etre',
 '',
 'fleurus',
 'cadex',
 'daralhadaiq',
 'blackbox',
 'goelette',
 'koine',
 'micmac',
 'bld',
 'samir',
 'hugo',
 'tsipika',
 'delcourt',
 'grund',
 '',
 'lemeac',
 'ndze',
 'daralhadaiq',
 'bartholdi',
 'h2t',
 'lagapante',
 '1018']

In [7]:
# assert(doublons == ['memo', 'ekare', 'denoel', 'le lievre de mars', 'palette', 'disneyhyperion books', 'petits genies', 'delachaux et niestle', 'la poule qui pond', 'motus', 'quatre fleuves', 'format', 'rue de lechiquier', 'eyrolles', 'ceres', 'pika', 'etre', 'grenouille', 'du beryl', 'ankama', 'du chene', 'dar alsaqi', 'grund', 'bamboo', 'dar aladab', 'edicef', 'du pepin', 'pemf', 'zebulo', 'etre', '', 'fleurus', 'cadex', 'dar alhadaiq', 'black box', 'goelette', 'koine', 'bld', 'samir', 'tsipika', '', 'lemeac', 'ndze', 'dar alhadaiq', 'bartholdi', 'h2t', '1018'])

In [8]:
import Levenshtein

def find_top_x_similar_strings_levenshtein(string_list, target_string):
    distances = []
    for string in string_list:
        distance = Levenshtein.distance(string, target_string)
        if distance <= 8:
            distances.append((string, distance))
    distances.sort(key=lambda x: x[1])
    if distances:
         return distances[:5]
    else: 
        return None

In [9]:
direct_match_counter = 0
levenstein_match_counter = 0
btlf_publishers_dict = {}
matches_dict = {}
non_matches_dict = {}
doublons_btlf = []
for publisher in publishers_btlf.subjects(RDF.type, SCHEMA.Publisher):
    raw_name = str(publishers_btlf.value(publisher, SCHEMA.name))
    preprocessed_name = utils.preprocess_publisher_name(raw_name)

    if preprocessed_name in btlf_publishers_dict.keys():
        doublons_btlf.append(preprocessed_name)
        
    btlf_publishers_dict[preprocessed_name] = utils.Publisher(source="btlf", 
                                                uri=publisher, 
                                                raw_name=raw_name, 
                                                preprocessed_name=preprocessed_name)

    if preprocessed_name in publishers_bnf_constellations_dict.keys():
        direct_match_counter +=1
        # print(preprocessed_name)
        matches_dict[preprocessed_name] = [btlf_publishers_dict[preprocessed_name], publishers_bnf_constellations_dict[preprocessed_name]]
    else:
        matched_key = find_top_x_similar_strings_levenshtein(publishers_bnf_constellations_dict.keys(), preprocessed_name)
        if matched_key:
            levenstein_match_counter += 1
            print("\""+ raw_name+ "\"")
            for key in matched_key:
                print("  ", publishers_bnf_constellations_dict[key[0]].uri.replace("http://schema.org/", " "))
        else:
            non_matches_dict[preprocessed_name] = publisher
print()
print("direct", direct_match_counter, "/", len(btlf_publishers_dict.keys()))
print("levenstein", levenstein_match_counter, "/", len(btlf_publishers_dict.keys()))
print("sum = ", direct_match_counter + levenstein_match_counter)

"Magnard-Vuibert"
    Magnard
    Magnard_Jeunesse
    LAudibert
    Lagoa_Editora
    Magnani
"L'Atelier"
    De_LAtelier
    Atelierddition
    Hatier
    Atelier_10
    LAmitié
"Musée du Louvre"
    MuseduLouvred
    Musée_du_Québec
    MuseDapper
    MusedOrsay
    Rue_Du_Monde
"Soréda"
    Boréal
    Orca
    Soleil
    Arena
    Ohd
"Hoche communication"
    Carrefour_Communication
    Hachetteducation
"Soleil productions"
    Soleilproduction
    Marsu_productions
    Top_Shelf_Productions
    Zproduction
    Papalunaproductions
"Dexia Editions"
    Sépia
    DEux
    Dada
    David
    Fei
"Les Editions du Quotidien"
    Druide
    Du_Centurion
    Didier
    Du_Sentier
    Du_Nordir
"Epsilon éditions"
    Ypsilon
    EpsilonBD
    Plon
    Siloë
    pilobeditions
"Gulf Stream (Editions du)"
    Gulf_Stream
    Australditions
    Rusticad
    CultureSud
    Calligram
"Cyr éditions"
    XYZ_Éditeur
    CPP
    CK
    CNT
    ACR
"Grain de sable"
    Grainsdesel
    Théâtre_De_Sa

In [10]:
len(non_matches_dict)

42

In [11]:
non_matches_dict

{'alsegpresseetmultimedia': rdflib.term.URIRef('http://schema.org/publisher116'),
 'zuidnederlandseuitgeverij': rdflib.term.URIRef('http://schema.org/publisher152'),
 'laptitescenequibouge': rdflib.term.URIRef('http://schema.org/publisher196'),
 'associationculturellefranceamerique': rdflib.term.URIRef('http://schema.org/publisher223'),
 'galeriebarbier&mathon': rdflib.term.URIRef('http://schema.org/publisher225'),
 'mediaparticipationsparis': rdflib.term.URIRef('http://schema.org/publisher23'),
 'donneravoirassociation': rdflib.term.URIRef('http://schema.org/publisher249'),
 'societenouvelleannecarriere': rdflib.term.URIRef('http://schema.org/publisher260'),
 'centrefrancelivres': rdflib.term.URIRef('http://schema.org/publisher295'),
 'onamarchesurlabulle': rdflib.term.URIRef('http://schema.org/publisher299'),
 'bibliothequelecturedeveloppement': rdflib.term.URIRef('http://schema.org/publisher310'),
 'radiofranceinternationale': rdflib.term.URIRef('http://schema.org/publisher322'),
 '